In [3]:
!pip install graphviz

In [4]:
# ===================================================================
# APLICACIÓN TP N°2 - MÁQUINAS MARINAS III
# (Versión Refactorizada con Gantt y Red PERT)
# ===================================================================
from dataclasses import dataclass
import math
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from IPython.display import display, clear_output, HTML
import markdown # Requerido para el reporte markdown

# --- NUEVAS IMPORTACIONES PARA GRÁFICOS ---
import plotly.graph_objects as go
import pandas as pd
try:
    from graphviz import Digraph
except ImportError:
    print("ADVERTENCIA: Librería 'graphviz' no encontrada. El diagrama de red no funcionará.")
    print("Por favor, ejecute en una celda: !pip install graphviz")
    Digraph = None # Para evitar que el código falle si no está instalada

# -------------------------------------------------------------------
# 0. FUNCIONES DE UTILIDAD
# -------------------------------------------------------------------

def check_str(cumple: bool) -> str:
    """Retorna HTML 'CUMPLE' (verde) o 'NO CUMPLE' (rojo)."""
    if cumple: return "<span style='color:green; font-weight:bold;'>CUMPLE</span>"
    else: return "<span style='color:red; font-weight:bold;'>NO CUMPLE</span>"

def balance_str(balance: float) -> str:
    """Retorna HTML para superávit (verde) o déficit (rojo)."""
    if balance >= 0: return f"<span style='color:green;'>SUPERÁVIT: {balance:.2f} m³/día</span>"
    else: return f"<span style='color:red;'>DÉFICIT: {balance:.2f} m³/día</span>"

# -------------------------------------------------------------------
# 1. CLASES DE CONFIGURACIÓN (INPUTS)
# -------------------------------------------------------------------
# (Definiciones de Dataclass idénticas)

@dataclass
class MotorPrincipalConfig:
    potencia_mcr_kw: float; tipo_combustible: str; sfoc_hfo_g_kwh: float
    sfoc_mdo_g_kwh: float; factor_carga_navegacion: float; vol_barrido_total_m3: float
@dataclass
class MotorAuxiliarConfig:
    potencia_kw: float; sfoc_g_kwh: float; numero_total: int
    en_servicio_navegacion: int; factor_carga_navegacion: float; vol_barrido_total_m3: float
@dataclass
class CalderaConfig:
    consumo_kg_h: float; tipo_combustible: str; sfoc_hfo_g_kgh: float
    sfoc_mdo_g_kgh: float; sfoc_mgo_g_kgh: float; en_navegacion: bool
@dataclass
class TanquesCombustibleConfig:
    almacenamiento_hfo_m3: float; almacenamiento_mdo_m3: float
    sedimentacion_hfo_m3: float; sedimentacion_mdo_m3: float
    diario_hfo_m3: float; diario_mdo_m3: float
@dataclass
class CriteriosLubricacionConfig:
    carter_mpp_litros_por_kw: float; capacidad_carter_mmaa_m3: float
    almac_num_cambios_mpp: float; almac_num_cambios_mmaa: float
    sucio_num_cambios_mpp: float; sucio_num_cambios_mmaa: float
@dataclass
class TanquesLubricacionConfig:
    circulacion_mpp_m3: float; almacenamiento_mpp_m3: float
    almacenamiento_mmaa_m3: float; sucio_mpp_m3: float; sucio_mmaa_m3: float
@dataclass
class CriteriosAguaDulceConfig:
    tripulacion_numero: int; dotacion_potable_litros_persona_dia: float
    autonomia_reserva_potable_dias: float; reserva_tecnica_operativa_m3: float
@dataclass
class SistemaAguaDulceConfig:
    capacidad_generador_agua_m3_dia: float; volumen_tanques_agua_dulce_m3: float
@dataclass
class CriteriosAireComprimidoConfig:
    mpp_num_arranques: int; mmaa_num_arranques: int; factor_consumo_mpp_K: float
    factor_consumo_mmaa_K: float; presion_max_bar: float; presion_min_mpp_bar: float
    presion_min_mmaa_bar: float
@dataclass
class SistemaAireComprimidoConfig:
    proyecto_num_botellones: int; proyecto_vol_botellon_m3: float
@dataclass
class CriteriosAchiqueConfig:
    coeficiente_k1_hfo: float; coeficiente_k1_mdo: float; autonomia_viaje_dias: int
@dataclass
class SistemaAchiqueConfig:
    proyecto_vol_tanque_fangos_m3: float; proyecto_vol_tanque_sentina_m3: float
@dataclass
class BuqueConfig:
    mpp: MotorPrincipalConfig; mmaa: MotorAuxiliarConfig; caldera: CalderaConfig
    tanques_comb: TanquesCombustibleConfig; densidad_hfo_t_m3: float; densidad_mdo_t_m3: float
    tanques_lub: TanquesLubricacionConfig; criterios_lub: CriteriosLubricacionConfig
    criterios_agua: CriteriosAguaDulceConfig; sistema_agua: SistemaAguaDulceConfig
    criterios_aire: CriteriosAireComprimidoConfig; sistema_aire: SistemaAireComprimidoConfig
    criterios_achique: CriteriosAchiqueConfig; sistema_achique: SistemaAchiqueConfig


# -------------------------------------------------------------------
# 2. CLASES DE ANÁLISIS (PROCESSING ENGINE)
# -------------------------------------------------------------------
# (Las 5 clases de análisis: Combustible, Lubricacion, AguaDulce,
#  AireComprimido, y Achique, permanecen idénticas a la versión anterior.
#  Incluyen los métodos _init_, _calcular_, generar_reporte, y generar_markdown)

# --- Motor de Cálculo: Módulo A (Combustible) ---
class AnalisisSistemaCombustible:
    def __init__(self, config: BuqueConfig):
        self.config = config; self.CRITERIO_HORAS_SEDIMENTACION = 24; self.CRITERIO_HORAS_DIARIO = 8
        self.consumos_diarios = {}; self.analisis_autonomia = {}; self.verificacion_tanques = {}
        if config.densidad_hfo_t_m3 <= 0 or config.densidad_mdo_t_m3 <= 0: raise ValueError("Las densidades del combustible deben ser positivas.")
        self._calcular_consumos_diarios(); self._calcular_autonomia(); self._verificar_tanques_reglamentarios()
    def _calcular_consumos_diarios(self):
        cfg_mpp = self.config.mpp; cfg_mmaa = self.config.mmaa; cfg_caldera = self.config.caldera
        potencia_servicio_mpp = cfg_mpp.potencia_mcr_kw * cfg_mpp.factor_carga_navegacion
        consumo_mpp_hfo_t_dia = 0; consumo_mpp_mdo_t_dia = 0; sfoc_mpp_usado = 0
        if cfg_mpp.tipo_combustible == 'HFO':
            sfoc_mpp_usado = cfg_mpp.sfoc_hfo_g_kwh; consumo_mpp_hfo_t_dia = (potencia_servicio_mpp * sfoc_mpp_usado * 24) / 1_000_000
        else: sfoc_mpp_usado = cfg_mpp.sfoc_mdo_g_kwh; consumo_mpp_mdo_t_dia = (potencia_servicio_mpp * sfoc_mpp_usado * 24) / 1_000_000
        consumo_caldera_hfo_t_dia = 0; consumo_caldera_mdo_t_dia = 0; consumo_caldera_mgo_t_dia = 0
        if cfg_caldera.en_navegacion:
            consumo_caldera_t_dia = (cfg_caldera.consumo_kg_h * 24) / 1000
            if cfg_caldera.tipo_combustible == 'HFO': consumo_caldera_hfo_t_dia = consumo_caldera_t_dia
            elif cfg_caldera.tipo_combustible == 'MDO': consumo_caldera_mdo_t_dia = consumo_caldera_t_dia
            elif cfg_caldera.tipo_combustible == 'MGO': consumo_caldera_mgo_t_dia = consumo_caldera_t_dia
        potencia_servicio_mmaa = (cfg_mmaa.potencia_kw * cfg_mmaa.en_servicio_navegacion * cfg_mmaa.factor_carga_navegacion)
        consumo_mmaa_t_dia = (potencia_servicio_mmaa * cfg_mmaa.sfoc_g_kwh * 24) / 1_000_000
        total_hfo_t_dia = consumo_mpp_hfo_t_dia + consumo_caldera_hfo_t_dia
        total_mdo_t_dia = consumo_mpp_mdo_t_dia + consumo_mmaa_t_dia + consumo_caldera_mdo_t_dia
        total_mgo_t_dia = consumo_caldera_mgo_t_dia
        self.consumos_diarios = {
            "consumo_hfo_t_dia": total_hfo_t_dia, "consumo_mdo_t_dia": total_mdo_t_dia, "consumo_mgo_t_dia": total_mgo_t_dia,
            "detalle": {"potencia_servicio_mpp_kw": potencia_servicio_mpp, "mpp_usa_comb": cfg_mpp.tipo_combustible, "mpp_sfoc_usado": sfoc_mpp_usado, "consumo_mpp_hfo": consumo_mpp_hfo_t_dia, "consumo_mpp_mdo": consumo_mpp_mdo_t_dia, "caldera_usa_comb": cfg_caldera.tipo_combustible, "consumo_caldera_hfo": consumo_caldera_hfo_t_dia, "consumo_caldera_mdo": consumo_caldera_mdo_t_dia, "consumo_caldera_mgo": consumo_caldera_mgo_t_dia, "potencia_servicio_mmaa_kw": potencia_servicio_mmaa, "consumo_mmaa_mdo": consumo_mmaa_t_dia}
        }
    def _calcular_autonomia(self):
        cfg_tanques = self.config.tanques_comb; masa_hfo = cfg_tanques.almacenamiento_hfo_m3 * self.config.densidad_hfo_t_m3; masa_mdo = cfg_tanques.almacenamiento_mdo_m3 * self.config.densidad_mdo_t_m3
        consumo_hfo = self.consumos_diarios["consumo_hfo_t_dia"]; consumo_mdo_mgo_total = self.consumos_diarios["consumo_mdo_t_dia"] + self.consumos_diarios["consumo_mgo_t_dia"]
        autonomia_hfo = (masa_hfo / consumo_hfo) if consumo_hfo > 0 else float('inf'); autonomia_mdo_mgo = (masa_mdo / consumo_mdo_mgo_total) if consumo_mdo_mgo_total > 0 else float('inf')
        autonomia_total = min(autonomia_hfo, autonomia_mdo_mgo); limitante = 'N/A'
        if autonomia_total == float('inf'): autonomia_total = 0
        elif autonomia_hfo <= autonomia_mdo_mgo: limitante = 'HFO'
        else: limitante = 'MDO/MGO'
        self.analisis_autonomia = {"autonomia_total_dias": autonomia_total, "limitante": limitante, "detalle": { "masa_hfo_t": masa_hfo, "masa_mdo_t": masa_mdo, "autonomia_hfo_dias": autonomia_hfo, "autonomia_mdo_mgo_dias": autonomia_mdo_mgo }}
    def _verificar_tanques_reglamentarios(self):
        cfg_tanques = self.config.tanques_comb; consumos = self.consumos_diarios; consumo_hfo_t_dia = consumos["consumo_hfo_t_dia"]; consumo_mdo_mgo_t_dia = consumos["consumo_mdo_t_dia"] + consumos["consumo_mgo_t_dia"]
        vol_sed_hfo_req = (consumo_hfo_t_dia / self.config.densidad_hfo_t_m3) * (self.CRITERIO_HORAS_SEDIMENTACION / 24); vol_diario_hfo_req = (consumo_hfo_t_dia / self.config.densidad_hfo_t_m3) * (self.CRITERIO_HORAS_DIARIO / 24)
        vol_sed_mdo_req = (consumo_mdo_mgo_t_dia / self.config.densidad_mdo_t_m3) * (self.CRITERIO_HORAS_SEDIMENTACION / 24); vol_diario_mdo_req = (consumo_mdo_mgo_t_dia / self.config.densidad_mdo_t_m3) * (self.CRITERIO_HORAS_DIARIO / 24)
        self.verificacion_tanques = {
            "hfo": {"sedimentacion": {"requerido_m3": vol_sed_hfo_req, "proyecto_m3": cfg_tanques.sedimentacion_hfo_m3, "cumple": cfg_tanques.sedimentacion_hfo_m3 >= vol_sed_hfo_req}, "diario": {"requerido_m3": vol_diario_hfo_req, "proyecto_m3": cfg_tanques.diario_hfo_m3, "cumple": cfg_tanques.diario_hfo_m3 >= vol_diario_hfo_req}},
            "mdo_mgo": {"sedimentacion": {"requerido_m3": vol_sed_mdo_req, "proyecto_m3": cfg_tanques.sedimentacion_mdo_m3, "cumple": cfg_tanques.sedimentacion_mdo_m3 >= vol_sed_mdo_req}, "diario": {"requerido_m3": vol_diario_mdo_req, "proyecto_m3": cfg_tanques.diario_mdo_m3, "cumple": cfg_tanques.diario_mdo_m3 >= vol_diario_mdo_req}}
        }
    def generar_reporte(self):
        consumos = self.consumos_diarios; autonomia = self.analisis_autonomia; verificacion = self.verificacion_tanques
        print("=======================================================================\n REPORTE DE ANÁLISIS: SISTEMA DE COMBUSTIBLE (BLOQUE A)\n=======================================================================\n\n--- A.1. Perfil de Consumo (Navegación Franca) ---")
        print(f"  Combustible MMPP seleccionado:  {consumos['detalle']['mpp_usa_comb']} (SFOC: {consumos['detalle']['mpp_sfoc_usado']:.2f} g/kWh)")
        print(f"  Combustible Caldera seleccionado: {consumos['detalle']['caldera_usa_comb']}")
        print(f"  Consumo HFO (MMPP + Caldera): {consumos['consumo_hfo_t_dia']:.2f} t/día\n  Consumo MDO (MMPP + MMAA + Caldera): {consumos['consumo_mdo_t_dia']:.2f} t/día")
        print(f"  Consumo MGO (Caldera): {consumos.get('consumo_mgo_t_dia', 0):.2f} t/día\n\n--- A.2. Cálculo de Autonomía ---")
        print(f"  Autonomía (limitante): {autonomia['autonomia_total_dias']:.2f} días\n  **Combustible limitante: {autonomia['limitante']}**\n\n--- A.3 y A.4. Verificación de Tanques ---")
        print(f"  Criterios: Sedimentación ({self.CRITERIO_HORAS_SEDIMENTACION}h), Diario ({self.CRITERIO_HORAS_DIARIO}h)")
        v_sed_hfo = verificacion['hfo']['sedimentacion']; v_dia_hfo = verificacion['hfo']['diario']
        print(f"\n  HFO:\n    Tanque Sedimentación: Req: {v_sed_hfo['requerido_m3']:.2f} m³ | Proyecto: {v_sed_hfo['proyecto_m3']:.2f} m³ | {check_str(v_sed_hfo['cumple'])}")
        print(f"    Tanque Diario:        Req: {v_dia_hfo['requerido_m3']:.2f} m³ | Proyecto: {v_dia_hfo['proyecto_m3']:.2f} m³ | {check_str(v_dia_hfo['cumple'])}")
        v_sed_mdo_mgo = verificacion['mdo_mgo']['sedimentacion']; v_dia_mdo_mgo = verificacion['mdo_mgo']['diario']
        print(f"\n  MDO/MGO (Tanques MDO):\n    Tanque Sedimentación: Req: {v_sed_mdo_mgo['requerido_m3']:.2f} m³ | Proyecto: {v_sed_mdo_mgo['proyecto_m3']:.2f} m³ | {check_str(v_sed_mdo_mgo['cumple'])}")
        print(f"    Tanque Diario:        Req: {v_dia_mdo_mgo['requerido_m3']:.2f} m³ | Proyecto: {v_dia_mdo_mgo['proyecto_m3']:.2f} m³ | {check_str(v_dia_mdo_mgo['cumple'])}\n=======================================================================")
    def generar_markdown(self) -> str:
        autonomia = self.analisis_autonomia; verificacion = self.verificacion_tanques
        md = "## 📈 Bloque A: Sistema de Combustible\n\n| Criterio | Resultado |\n| :--- | :--- |\n"
        md += f"| **Autonomía Total** | {autonomia['autonomia_total_dias']:.2f} días |\n| **Combustible Limitante** | **{autonomia['limitante']}** |\n"
        md += f"| Tq. Sedim. HFO ({self.CRITERIO_HORAS_SEDIMENTACION}h) | {check_str(verificacion['hfo']['sedimentacion']['cumple'])} |\n| Tq. Diario HFO ({self.CRITERIO_HORAS_DIARIO}h) | {check_str(verificacion['hfo']['diario']['cumple'])} |\n"
        md += f"| Tq. Sedim. MDO/MGO ({self.CRITERIO_HORAS_SEDIMENTACION}h) | {check_str(verificacion['mdo_mgo']['sedimentacion']['cumple'])} |\n| Tq. Diario MDO/MGO ({self.CRITERIO_HORAS_DIARIO}h) | {check_str(verificacion['mdo_mgo']['diario']['cumple'])} |\n"
        return md
# --- Motor de Cálculo: Módulo B (Lubricación) ---
class AnalisisSistemaLubricacion:
    def __init__(self, config: BuqueConfig):
        self.config = config; self.resultados = {}; self.vol_base_carter_mpp_m3 = 0
        self._verificar_carter_mpp(); self._verificar_almacenamiento(); self._verificar_aceite_sucio()
    def _verificar_carter_mpp(self):
        cfg_mpp = self.config.mpp; cfg_crit = self.config.criterios_lub; cfg_tanq = self.config.tanques_lub
        vol_req_m3 = (cfg_mpp.potencia_mcr_kw * cfg_crit.carter_mpp_litros_por_kw) / 1000
        self.resultados['carter_mpp'] = {"requerido_m3": vol_req_m3, "proyecto_m3": cfg_tanq.circulacion_mpp_m3, "cumple": cfg_tanq.circulacion_mpp_m3 >= vol_req_m3}
        self.vol_base_carter_mpp_m3 = max(vol_req_m3, cfg_tanq.circulacion_mpp_m3)
    def _verificar_almacenamiento(self):
        cfg_mmaa = self.config.mmaa; cfg_crit = self.config.criterios_lub; cfg_tanq = self.config.tanques_lub
        vol_req_mpp = self.vol_base_carter_mpp_m3 * cfg_crit.almac_num_cambios_mpp; vol_carter_total_mmaa = cfg_crit.capacidad_carter_mmaa_m3 * cfg_mmaa.numero_total
        vol_req_mmaa = vol_carter_total_mmaa * cfg_crit.almac_num_cambios_mmaa
        self.resultados['almacenamiento'] = {"mpp": {"requerido_m3": vol_req_mpp, "proyecto_m3": cfg_tanq.almacenamiento_mpp_m3, "cumple": cfg_tanq.almacenamiento_mpp_m3 >= vol_req_mpp}, "mmaa": {"requerido_m3": vol_req_mmaa, "proyecto_m3": cfg_tanq.almacenamiento_mmaa_m3, "cumple": cfg_tanq.almacenamiento_mmaa_m3 >= vol_req_mmaa}}
    def _verificar_aceite_sucio(self):
        cfg_mmaa = self.config.mmaa; cfg_crit = self.config.criterios_lub; cfg_tanq = self.config.tanques_lub
        vol_req_mpp = self.vol_base_carter_mpp_m3 * cfg_crit.sucio_num_cambios_mpp; vol_carter_total_mmaa = cfg_crit.capacidad_carter_mmaa_m3 * cfg_mmaa.numero_total
        vol_req_mmaa = vol_carter_total_mmaa * cfg_crit.sucio_num_cambios_mmaa
        self.resultados['sucio'] = {"mpp": {"requerido_m3": vol_req_mpp, "proyecto_m3": cfg_tanq.sucio_mpp_m3, "cumple": cfg_tanq.sucio_mpp_m3 >= vol_req_mpp}, "mmaa": {"requerido_m3": vol_req_mmaa, "proyecto_m3": cfg_tanq.sucio_mmaa_m3, "cumple": cfg_tanq.sucio_mmaa_m3 >= vol_req_mmaa}}
    def generar_reporte(self):
        res = self.resultados; crit = self.config.criterios_lub
        print("\n\n=======================================================================\n REPORTE DE ANÁLISIS: SISTEMA DE LUBRICACIÓN (BLOQUE B)\n=======================================================================\n\n--- B.2. Verificación Tanque Circulación (Cárter) MMPP ---")
        print(f"  Criterio: {crit.carter_mpp_litros_por_kw} L/kW"); v_carter = res['carter_mpp']
        print(f"  Vol. Cárter MMPP: Req: {v_carter['requerido_m3']:.2f} m³ | Proyecto: {v_carter['proyecto_m3']:.2f} m³ | {check_str(v_carter['cumple'])}\n\n--- B.3. Verificación Tanques de Almacenamiento ---")
        print(f"  Criterio MMPP: {crit.almac_num_cambios_mpp} cambio(s) de {self.vol_base_carter_mpp_m3:.2f} m³"); print(f"  Criterio MMAA: {crit.almac_num_cambios_mmaa} cambio(s) de {self.config.mmaa.numero_total}x {crit.capacidad_carter_mmaa_m3} m³")
        v_almac_mpp = res['almacenamiento']['mpp']; v_almac_mmaa = res['almacenamiento']['mmaa']
        print(f"  Almac. MMPP: Req: {v_almac_mpp['requerido_m3']:.2f} m³ | Proyecto: {v_almac_mpp['proyecto_m3']:.2f} m³ | {check_str(v_almac_mpp['cumple'])}"); print(f"  Almac. MMAA: Req: {v_almac_mmaa['requerido_m3']:.2f} m³ | Proyecto: {v_almac_mmaa['proyecto_m3']:.2f} m³ | {check_str(v_almac_mmaa['cumple'])}\n\n--- B.4. Verificación Tanques de Aceite Sucio ---")
        print(f"  Criterio MMPP: {crit.sucio_num_cambios_mpp} cambio(s) de {self.vol_base_carter_mpp_m3:.2f} m³"); print(f"  Criterio MMAA: {crit.sucio_num_cambios_mmaa} cambio(s) de {self.config.mmaa.numero_total}x {crit.capacidad_carter_mmaa_m3} m³")
        v_sucio_mpp = res['sucio']['mpp']; v_sucio_mmaa = res['sucio']['mmaa']
        print(f"  Sucio MMPP:  Req: {v_sucio_mpp['requerido_m3']:.2f} m³ | Proyecto: {v_sucio_mpp['proyecto_m3']:.2f} m³ | {check_str(v_sucio_mpp['cumple'])}"); print(f"  Sucio MMAA:  Req: {v_sucio_mmaa['requerido_m3']:.2f} m³ | Proyecto: {v_sucio_mmaa['proyecto_m3']:.2f} m³ | {check_str(v_sucio_mmaa['cumple'])}\n=======================================================================")
    def generar_markdown(self) -> str:
        res = self.resultados; md = "\n## 💧 Bloque B: Sistema de Lubricación\n\n| Criterio | Resultado |\n| :--- | :--- |\n"
        md += f"| Tq. Circulación MMPP | {check_str(res['carter_mpp']['cumple'])} |\n| Tq. Almacenamiento MMPP | {check_str(res['almacenamiento']['mpp']['cumple'])} |\n"
        md += f"| Tq. Almacenamiento MMAA | {check_str(res['almacenamiento']['mmaa']['cumple'])} |\n| Tq. Aceite Sucio MMPP | {check_str(res['sucio']['mpp']['cumple'])} |\n"
        md += f"| Tq. Aceite Sucio MMAA | {check_str(res['sucio']['mmaa']['cumple'])} |\n"; return md
# --- Motor de Cálculo: Módulo C (Agua Dulce) ---
class AnalisisSistemaAguaDulce:
    def __init__(self, config: BuqueConfig):
        self.config = config; self.resultados = {}; self._calcular_volumen_requerido(); self._verificar_balance_diario()
    def _calcular_volumen_requerido(self):
        cfg_crit = self.config.criterios_agua; cfg_sis = self.config.sistema_agua; vol_potable_req_m3 = (cfg_crit.tripulacion_numero * cfg_crit.dotacion_potable_litros_persona_dia * cfg_crit.autonomia_reserva_potable_dias) / 1000
        vol_tecnica_req_m3 = cfg_crit.reserva_tecnica_operativa_m3; vol_total_req_m3 = vol_potable_req_m3 + vol_tecnica_req_m3
        self.resultados['volumen_tanque'] = {"requerido_m3": vol_total_req_m3, "proyecto_m3": cfg_sis.volumen_tanques_agua_dulce_m3, "cumple": cfg_sis.volumen_tanques_agua_dulce_m3 >= vol_total_req_m3, "detalle": {"potable_req_m3": vol_potable_req_m3, "tecnica_req_m3": vol_tecnica_req_m3}}
    def _verificar_balance_diario(self):
        cfg_crit = self.config.criterios_agua; cfg_sis = self.config.sistema_agua; consumo_diario_m3_dia = (cfg_crit.tripulacion_numero * cfg_crit.dotacion_potable_litros_persona_dia) / 1000
        generacion_diaria_m3_dia = cfg_sis.capacidad_generador_agua_m3_dia; balance_diario_m3_dia = generacion_diaria_m3_dia - consumo_diario_m3_dia
        self.resultados['balance_diario'] = {"consumo_m3_dia": consumo_diario_m3_dia, "generacion_m3_dia": generacion_diaria_m3_dia, "balance_m3_dia": balance_diario_m3_dia, "suficiente": balance_diario_m3_dia >= 0}
    def generar_reporte(self):
        res_vol = self.resultados['volumen_tanque']; res_bal = self.resultados['balance_diario']; crit = self.config.criterios_agua
        print("\n\n=======================================================================\n REPORTE DE ANÁLISIS: SISTEMA DE AGUA DULCE (BLOQUE C)\n=======================================================================\n\n--- C.2. Verificación Volumen Tanques Agua Dulce ---")
        print(f"  Criterios:\n    - Reserva Potable: {crit.tripulacion_numero} trip. * {crit.dotacion_potable_litros_persona_dia} L/p/día * {crit.autonomia_reserva_potable_dias} días = {res_vol['detalle']['potable_req_m3']:.2f} m³")
        print(f"    - Reserva Técnica: {res_vol['detalle']['tecnica_req_m3']:.2f} m³\n  Resultado:\n    Vol. Total Req.: {res_vol['requerido_m3']:.2f} m³ | Proyecto: {res_vol['proyecto_m3']:.2f} m³ | {check_str(res_vol['cumple'])}")
        print("\n--- C.3. Verificación de Balance Diario (Generación) ---"); print(f"  Consumo Diario (Potable): {res_bal['consumo_m3_dia']:.2f} m³/día"); print(f"  Generación Diaria (FWG):  {res_bal['generacion_m3_dia']:.2f} m³/día")
        print(f"  **Balance de Generación:  {balance_str(res_bal['balance_m3_dia'])}**\n=======================================================================")
    def generar_markdown(self) -> str:
        res_vol = self.resultados['volumen_tanque']; res_bal = self.resultados['balance_diario']
        md = "\n## 🚰 Bloque C: Sistema de Agua Dulce\n\n| Criterio | Resultado |\n| :--- | :--- |\n"
        md += f"| Capacidad Tanques FW | {check_str(res_vol['cumple'])} (Req: {res_vol['requerido_m3']:.2f} m³) |\n"
        md += f"| Balance Diario FWG | {balance_str(res_bal['balance_m3_dia'])} |\n"; return md
# --- Motor de Cálculo: Módulo D (Aire Comprimido) ---
class AnalisisSistemaAireComprimido:
    def __init__(self, config: BuqueConfig):
        self.config = config; self.resultados = {}; self._calcular_volumenes_requeridos(); self._verificar_volumen_proyecto()
    def _calcular_volumenes_requeridos(self):
        cfg_mpp = self.config.mpp; cfg_mmaa = self.config.mmaa; cfg_crit = self.config.criterios_aire; vol_req_mpp = 0; vol_req_mmaa = 0
        try:
            if cfg_crit.presion_max_bar <= cfg_crit.presion_min_mpp_bar: raise ValueError("P_max debe ser > P_min MMPP")
            denominador_mpp = math.log(cfg_crit.presion_max_bar / cfg_crit.presion_min_mpp_bar); vol_req_mpp = (cfg_crit.mpp_num_arranques * cfg_crit.factor_consumo_mpp_K * cfg_mpp.vol_barrido_total_m3) / denominador_mpp
        except Exception as e: print(f"ADVERTENCIA (Cálculo Aire MMPP): {e}")
        try:
            if cfg_crit.presion_max_bar <= cfg_crit.presion_min_mmaa_bar: raise ValueError("P_max debe ser > P_min MMAA")
            denominador_mmaa = math.log(cfg_crit.presion_max_bar / cfg_crit.presion_min_mmaa_bar); vol_req_mmaa = (cfg_crit.mmaa_num_arranques * cfg_mmaa.numero_total * cfg_crit.factor_consumo_mmaa_K * cfg_mmaa.vol_barrido_total_m3) / denominador_mmaa
        except Exception as e: print(f"ADVERTENCIA (Cálculo Aire MMAA): {e}")
        vol_total_req_m3 = vol_req_mpp + vol_req_mmaa; self.resultados['volumen_requerido'] = {"total_m3": vol_total_req_m3, "detalle": {"mpp_m3": vol_req_mpp, "mmaa_m3": vol_req_mmaa}}
    def _verificar_volumen_proyecto(self):
        cfg_sis = self.config.sistema_aire; vol_req_total = self.resultados['volumen_requerido']['total_m3']; vol_proyecto_total = cfg_sis.proyecto_num_botellones * cfg_sis.proyecto_vol_botellon_m3
        self.resultados['verificacion_proyecto'] = {"requerido_m3": vol_req_total, "proyecto_m3": vol_proyecto_total, "cumple": vol_proyecto_total >= vol_req_total}
    def generar_reporte(self):
        res_req = self.resultados['volumen_requerido']; res_ver = self.resultados['verificacion_proyecto']; crit = self.config.criterios_aire; cfg_sis = self.config.sistema_aire
        print("\n\n=======================================================================\n REPORTE DE ANÁLISIS: SISTEMA DE AIRE COMPRIMIDO (BLOQUE D)\n=======================================================================\n\n--- D.2. Cálculo Volumen Requerido (MMPP) ---")
        print(f"  Criterio: {crit.mpp_num_arranques} arranques, K={crit.factor_consumo_mpp_K}, P_max/P_min={crit.presion_max_bar:.2f}/{crit.presion_min_mpp_bar:.2f} bar\n  Volumen requerido MMPP: {res_req['detalle']['mpp_m3']:.2f} m³")
        print(f"\n--- D.3. Cálculo Volumen Requerido (MMAA) ---\n  Criterio: {crit.mmaa_num_arranques} arranques c/u, K={crit.factor_consumo_mmaa_K}, P_max/P_min={crit.presion_max_bar:.2f}/{crit.presion_min_mmaa_bar:.2f} bar\n  Volumen requerido MMAA (total): {res_req['detalle']['mmaa_m3']:.2f} m³")
        print(f"\n--- D.4. Verificación Botellones de Arranque ---\n  Volumen Total Requerido (MMPP + MMAA): {res_ver['requerido_m3']:.2f} m³")
        print(f"  Volumen de Proyecto ({cfg_sis.proyecto_num_botellones} botellones * {cfg_sis.proyecto_vol_botellon_m3:.2f} m³ c/u): {res_ver['proyecto_m3']:.2f} m³\n  **Resultado: {check_str(res_ver['cumple'])}**\n=======================================================================")
    def generar_markdown(self) -> str:
        res_ver = self.resultados['verificacion_proyecto']
        md = "\n## 💨 Bloque D: Sistema de Aire Comprimido\n\n| Criterio | Resultado |\n| :--- | :--- |\n"
        md += f"| Capacidad Botellones (SOLAS) | {check_str(res_ver['cumple'])} (Req: {res_ver['requerido_m3']:.2f} m³) |\n"; return md
# --- Motor de Cálculo: Módulo E (Achique/Fangos) ---
class AnalisisSistemaAchique:
    def __init__(self, config: BuqueConfig, consumos_combustible: dict):
        self.config = config; self.consumos = consumos_combustible; self.resultados = {}; self._verificar_tanque_fangos()
    def _verificar_tanque_fangos(self):
        cfg_crit = self.config.criterios_achique; cfg_sis = self.config.sistema_achique; consumo_hfo = self.consumos.get("consumo_hfo_t_dia", 0)
        consumo_mdo_mgo = self.consumos.get("consumo_mdo_t_dia", 0) + self.consumos.get("consumo_mgo_t_dia", 0); prod_fangos_distillate = consumo_mdo_mgo * cfg_crit.coeficiente_k1_mdo
        prod_fangos_hfo = consumo_hfo * cfg_crit.coeficiente_k1_hfo; prod_total_diaria_m3_dia = prod_fangos_hfo + prod_fangos_distillate
        vol_req_m3 = prod_total_diaria_m3_dia * cfg_crit.autonomia_viaje_dias; vol_min_marpol = 2.0; vol_req_final = max(vol_req_m3, vol_min_marpol)
        self.resultados['tanque_fangos'] = {
            "requerido_m3": vol_req_final, "proyecto_m3": cfg_sis.proyecto_vol_tanque_fangos_m3, "cumple": cfg_sis.proyecto_vol_tanque_fangos_m3 >= vol_req_final,
            "detalle": {"prod_diaria_m3": prod_total_diaria_m3_dia, "consumo_hfo_t_dia": consumo_hfo, "consumo_mdo_mgo_t_dia": consumo_mdo_mgo, "vol_calculado_formula_m3": vol_req_m3, "vol_minimo_marpol_m3": vol_min_marpol}
        }
    def generar_reporte(self):
        res = self.resultados['tanque_fangos']; crit = self.config.criterios_achique
        print("\n\n=======================================================================\n REPORTE DE ANÁLISIS: SISTEMA DE ACHIQUE (BLOQUE E)\n=======================================================================\n\n--- E.2. Verificación Tanque de Fangos (Sludge) ---")
        print(f"  Criterio MARPOL: V = K1 * C * D (o mínimo regulatorio)\n    - K1 (HFO/Distillate): {crit.coeficiente_k1_hfo} / {crit.coeficiente_k1_mdo}")
        print(f"    - C (Consumo HFO/Distillate): {res['detalle']['consumo_hfo_t_dia']:.2f} t/día / {res['detalle']['consumo_mdo_mgo_t_dia']:.2f} t/día\n    - D (Autonomía): {crit.autonomia_viaje_dias} días")
        print(f"  Producción Diaria de Fangos: {res['detalle']['prod_diaria_m3']:.3f} m³/día\n  Volumen Calculado (Fórmula): {res['detalle']['vol_calculado_formula_m3']:.2f} m³")
        print(f"  Volumen Mínimo MARPOL: {res['detalle']['vol_minimo_marpol_m3']:.2f} m³")
        print(f"  Volumen Requerido (el mayor): {res['requerido_m3']:.2f} m³ | Proyecto: {res['proyecto_m3']:.2f} m³ | {check_str(res['cumple'])}\n=======================================================================")
    def generar_markdown(self) -> str:
        res = self.resultados['tanque_fangos']
        md = "\n## 🛢️ Bloque E: Sistema de Achique (Fangos)\n\n| Criterio | Resultado |\n| :--- | :--- |\n"
        md += f"| Capacidad Tq. Fangos (MARPOL) | {check_str(res['cumple'])} (Req: {res['requerido_m3']:.2f} m³) |\n"; return md


# ===================================================================
# 3. INTERFAZ DE USUARIO INTERACTIVA (INPUTS)
# ===================================================================

# --- Estilos y Layouts ---
style = {'description_width': '180px'}; layout = Layout(width='450px')
layout_l = Layout(width='500px'); style_l = {'description_width': '230px'}
gantt_layout_text = Layout(width='350px')
gantt_layout_num = Layout(width='100px')
gantt_layout_dep = Layout(width='200px')

# --- Pestañas A, B, C, D, E (Idénticas a la versión anterior) ---
# ... (código de widgets de pestañas A-E) ...
# --- Pestaña 0: Bloque A (Combustible) ---
w_mpp_potencia = widgets.FloatText(value=2500, description='Potencia MCR (kW):', style=style, layout=layout)
w_mpp_tipo_comb = widgets.Dropdown(options=['HFO', 'MDO'], value='HFO', description='Combustible MMPP:', style=style, layout=layout)
w_mpp_sfoc_hfo = widgets.FloatText(value=182, description='SFOC (HFO) (g/kWh):', style=style, layout=layout)
w_mpp_sfoc_mdo = widgets.FloatText(value=178, description='SFOC (MDO) (g/kWh):', style=style, layout=layout)
w_mpp_factor_carga = widgets.FloatSlider(value=0.85, min=0.1, max=1.0, step=0.01, description='Factor Carga Nav.:', style=style, layout=layout)
w_mpp_vol_barrido = widgets.FloatText(value=0.532, description='Vol. Barrido Total (m³):', style=style_l, layout=layout_l, disabled=False)
ui_mpp = VBox([w_mpp_potencia, w_mpp_tipo_comb, w_mpp_sfoc_hfo, w_mpp_sfoc_mdo, w_mpp_factor_carga, w_mpp_vol_barrido])
w_mmaa_potencia = widgets.FloatText(value=320, description='Potencia c/u (kW):', style=style, layout=layout)
w_mmaa_sfoc = widgets.FloatText(value=218, description='SFOC (MDO) (g/kWh):', style=style, layout=layout)
w_mmaa_numero = widgets.IntText(value=3, description='N° Total:', style=style, layout=layout)
w_mmaa_en_servicio = widgets.IntText(value=1, description='N° en Nav.:', style=style, layout=layout)
w_mmaa_factor_carga = widgets.FloatSlider(value=0.60, min=0.1, max=1.0, step=0.01, description='Factor Carga Nav.:', style=style, layout=layout)
w_mmaa_vol_barrido = widgets.FloatText(value=0.018, description='Vol. Barrido c/u (m³):', style=style_l, layout=layout_l, disabled=False)
ui_mmaa = VBox([w_mmaa_potencia, w_mmaa_sfoc, w_mmaa_numero, w_mmaa_en_servicio, w_mmaa_factor_carga, w_mmaa_vol_barrido])
w_caldera_consumo = widgets.FloatText(value=50, description='Consumo (kg/h):', style=style, layout=layout)
w_caldera_tipo_comb = widgets.Dropdown(options=['HFO', 'MDO', 'MGO'], value='HFO', description='Combustible Caldera:', style=style, layout=layout)
w_caldera_sfoc_hfo = widgets.FloatText(value=700, description='SFOC (HFO) (g/kg vapor):', style=style, layout=layout, disabled=True)
w_caldera_sfoc_mdo = widgets.FloatText(value=700, description='SFOC (MDO) (g/kg vapor):', style=style, layout=layout, disabled=True)
w_caldera_sfoc_mgo = widgets.FloatText(value=700, description='SFOC (MGO) (g/kg vapor):', style=style, layout=layout, disabled=True)
w_caldera_en_nav = widgets.Checkbox(value=True, description='Encendida en Nav.', style=style, layout=layout)
ui_caldera = VBox([w_caldera_consumo, w_caldera_tipo_comb, w_caldera_en_nav, w_caldera_sfoc_hfo, w_caldera_sfoc_mdo, w_caldera_sfoc_mgo])
w_tanque_alm_hfo = widgets.FloatText(value=265, description='Almacenamiento HFO (m³):', style=style, layout=layout); w_tanque_alm_mdo = widgets.FloatText(value=46, description='Almacenamiento MDO (m³):', style=style, layout=layout)
w_tanque_sed_hfo = widgets.FloatText(value=16, description='Sedimentación HFO (m³):', style=style, layout=layout); w_tanque_sed_mdo = widgets.FloatText(value=6, description='Sedimentación MDO (m³):', style=style, layout=layout)
w_tanque_dia_hfo = widgets.FloatText(value=8, description='Diario HFO (m³):', style=style, layout=layout); w_tanque_dia_mdo = widgets.FloatText(value=6, description='Diario MDO (m³):', style=style, layout=layout)
ui_tanques_comb = VBox([w_tanque_alm_hfo, w_tanque_alm_mdo, w_tanque_sed_hfo, w_tanque_sed_mdo, w_tanque_dia_hfo, w_tanque_dia_mdo])
w_densidad_hfo = widgets.FloatText(value=0.98, description='Densidad HFO (t/m³):', style=style, layout=layout); w_densidad_mdo = widgets.FloatText(value=0.89, description='Densidad MDO (t/m³):', style=style, layout=layout)
ui_densidades = VBox([w_densidad_hfo, w_densidad_mdo])
accordion_comb = widgets.Accordion(children=[ui_mpp, ui_mmaa, ui_caldera, ui_tanques_comb, ui_densidades])
accordion_comb.set_title(0, 'A.1: Motores'); accordion_comb.set_title(1, 'A.2: Motores Auxiliares'); accordion_comb.set_title(2, 'A.3: Caldera'); accordion_comb.set_title(3, 'A.4: Tanques de Combustible (Proyecto)'); accordion_comb.set_title(4, 'A.5: Parámetros Generales')
# --- Pestaña 1: Bloque B (Lubricación) ---
w_lub_crit_carter_L_kW = widgets.FloatText(value=1.0, description='Criterio Cárter (L/kW MMPP):', style=style, layout=layout); w_lub_crit_almac_mpp = widgets.FloatText(value=1.0, description='Criterio Almac. (N° Cambios MMPP):', style=style, layout=layout)
w_lub_crit_carter_mmaa_m3 = widgets.FloatText(value=0.5, description='Capacidad Cárter (m³/MMAA):', style=style, layout=layout); w_lub_crit_almac_mmaa = widgets.FloatText(value=2.0, description='Criterio Almac. (N° Cambios MMAA):', style=style, layout=layout)
w_lub_crit_sucio_mpp = widgets.FloatText(value=1.0, description='Criterio Sucio (N° Cambios MMPP):', style=style, layout=layout); w_lub_crit_sucio_mmaa = widgets.FloatText(value=1.0, description='Criterio Sucio (N° Cambios MMAA):', style=style, layout=layout)
ui_lub_criterios = VBox([w_lub_crit_carter_L_kW, w_lub_crit_carter_mmaa_m3, w_lub_crit_almac_mpp, w_lub_crit_almac_mmaa, w_lub_crit_sucio_mpp, w_lub_crit_sucio_mmaa])
w_lub_tanq_circ_mpp = widgets.FloatText(value=3.5, description='Circulación MMPP (m³):', style=style, layout=layout); w_lub_tanq_almac_mpp = widgets.FloatText(value=8.0, description='Almacenamiento MMPP (m³):', style=style, layout=layout)
w_lub_tanq_almac_mmaa = widgets.FloatText(value=4.0, description='Almacenamiento MMAA (m³):', style=style, layout=layout); w_lub_tanq_sucio_mpp = widgets.FloatText(value=8.0, description='Aceite Sucio MMPP (m³):', style=style, layout=layout)
w_lub_tanq_sucio_mmaa = widgets.FloatText(value=3.0, description='Aceite Sucio MMAA (m³):', style=style, layout=layout)
ui_lub_tanques = VBox([w_lub_tanq_circ_mpp, w_lub_tanq_almac_mpp, w_lub_tanq_almac_mmaa, w_lub_tanq_sucio_mpp, w_lub_tanq_sucio_mmaa])
accordion_lub = widgets.Accordion(children=[ui_lub_criterios, ui_lub_tanques])
accordion_lub.set_title(0, 'B.1: Criterios de Cálculo (Cátedra)'); accordion_lub.set_title(1, 'B.2: Tanques de Lubricación (Proyecto)')
# --- Pestaña 2: Bloque C (Agua Dulce) ---
w_agua_crit_tripulacion = widgets.IntText(value=15, description='N° Tripulantes:', style=style, layout=layout); w_agua_crit_dotacion = widgets.FloatText(value=100, description='Dotación (L/p/día):', style=style, layout=layout)
w_agua_crit_autonomia = widgets.FloatText(value=3.0, description='Autonomía Reserva (días):', style=style, layout=layout); w_agua_crit_reserva_tec = widgets.FloatText(value=5.0, description='Reserva Técnica (m³):', style=style, layout=layout)
ui_agua_criterios = VBox([w_agua_crit_tripulacion, w_agua_crit_dotacion, w_agua_crit_autonomia, w_agua_crit_reserva_tec])
w_agua_proy_tanques = widgets.FloatText(value=100, description='Volumen Tanques FW (m³):', style=style, layout=layout); w_agua_proy_generador = widgets.FloatText(value=10, description='Capacidad FWG (m³/día):', style=style, layout=layout)
ui_agua_proyecto = VBox([w_agua_proy_tanques, w_agua_proy_generador])
accordion_agua = widgets.Accordion(children=[ui_agua_criterios, ui_agua_proyecto])
accordion_agua.set_title(0, 'C.1: Criterios de Cálculo (Cátedra)'); accordion_agua.set_title(1, 'C.2: Sistema de Agua Dulce (Proyecto)')
# --- Pestaña 3: Bloque D (Aire Comprimido) ---
w_aire_crit_mpp_arr = widgets.IntText(value=12, description='N° Arranques MMPP (SOLAS):', style=style_l, layout=layout_l); w_aire_crit_mmaa_arr = widgets.IntText(value=3, description='N° Arranques por MMAA (SOLAS):', style=style_l, layout=layout_l)
w_aire_crit_k_mpp = widgets.FloatText(value=8.0, description='Factor Consumo K (MMPP):', style=style_l, layout=layout_l); w_aire_crit_k_mmaa = widgets.FloatText(value=8.0, description='Factor Consumo K (MMAA):', style=style_l, layout=layout_l)
w_aire_crit_pmax = widgets.FloatText(value=30.0, description='Presión Máx. Botellones (bar):', style=style_l, layout=layout_l); w_aire_crit_pmin_mpp = widgets.FloatText(value=15.0, description='Presión Mín. Arranque MMPP (bar):', style=style_l, layout=layout_l)
w_aire_crit_pmin_mmaa = widgets.FloatText(value=10.0, description='Presión Mín. Arranque MMAA (bar):', style=style_l, layout=layout_l)
ui_aire_criterios = VBox([w_aire_crit_mpp_arr, w_aire_crit_mmaa_arr, w_aire_crit_k_mpp, w_aire_crit_k_mmaa, w_aire_crit_pmax, w_aire_crit_pmin_mpp, w_aire_crit_pmin_mmaa])
w_aire_proy_num = widgets.IntText(value=2, description='N° Botellones (Proyecto):', style=style_l, layout=layout_l); w_aire_proy_vol = widgets.FloatText(value=2.0, description='Volumen por Botellón (m³):', style=style_l, layout=layout_l)
ui_aire_proyecto = VBox([w_aire_proy_num, w_aire_proy_vol])
accordion_aire = widgets.Accordion(children=[ui_aire_criterios, ui_aire_proyecto])
accordion_aire.set_title(0, 'D.1: Criterios de Cálculo (Cátedra/SOLAS)'); accordion_aire.set_title(1, 'D.2: Sistema de Aire (Proyecto)')
# --- Pestaña 4: Bloque E (Achique/Fangos) ---
w_achique_crit_k1_hfo = widgets.FloatText(value=0.01, description='Coef. K1 (HFO) (MARPOL):', style=style_l, layout=layout_l)
w_achique_crit_k1_mdo = widgets.FloatText(value=0.005, description='Coef. K1 (MDO) (MARPOL):', style=style_l, layout=layout_l)
w_achique_crit_dias = widgets.IntText(value=30, description='Autonomía Viaje (Días):', style=style_l, layout=layout_l)
ui_achique_criterios = VBox([w_achique_crit_k1_hfo, w_achique_crit_k1_mdo, w_achique_crit_dias])
w_achique_proy_fangos = widgets.FloatText(value=8.0, description='Vol. Tanque Fangos (m³):', style=style_l, layout=layout_l)
w_achique_proy_sentina = widgets.FloatText(value=8.0, description='Vol. Tanque Sentina (m³):', style=style_l, layout=layout_l)
ui_achique_proyecto = VBox([w_achique_proy_fangos, w_achique_proy_sentina])
accordion_achique = widgets.Accordion(children=[ui_achique_criterios, ui_achique_proyecto])
accordion_achique.set_title(0, 'E.1: Criterios de Cálculo (MARPOL)'); accordion_achique.set_title(1, 'E.2: Sistema de Achique (Proyecto)')


# --- Pestaña 5: Bloque F (Secuencia de Arranque) ---

# (ID, Nombre, Duración, Dependencia_ID)
default_gantt_tasks = [
    (1, "1. EMER. DIESEL DRIVEN COMPRESSOR STARTED", 11, 0),
    (2, "2. AIR SUPPLY TO EMERG. AIR CYL. COMPLETED", 5, 0), # Paralela a 1
    (3, "3. MAIN DIESEL GEN. & MAIN MOTOR DRIVEN COMPRESSORS STARTED", 2, 2),
    (4, "4. AIR SUPPLY TO MAIN AIR CYL. COMPLETED", 21, 3),
    (5, "5. MAIN ENGINE STARTED", 2, 4),
    (6, "6. (Tarea Adicional)", 5, 5),
    (7, "7. (Tarea Adicional)", 5, 0),
    (8, "8. (Tarea Adicional)", 5, 0)
]

task_widgets = [] # Lista para guardar los widgets de cada fila
dependency_options = [("Inicio (Min 0)", 0)]
gantt_header = HBox([
    widgets.Label("Nombre de la Tarea", layout=gantt_layout_text),
    widgets.Label("Duración (min)", layout=gantt_layout_num),
    widgets.Label("Depende de:", layout=gantt_layout_dep)
])

for i in range(8):
    task_id, default_name, default_duration, default_dep_id = default_gantt_tasks[i]
    w_task_name = widgets.Text(value=default_name, layout=gantt_layout_text)
    w_task_duration = widgets.IntText(value=default_duration, layout=gantt_layout_num)
    w_task_dependency = widgets.Dropdown(options=dependency_options, value=default_dep_id, layout=gantt_layout_dep)
    task_widgets.append({"id": task_id, "name": w_task_name, "duration": w_task_duration, "dependency": w_task_dependency})
    dependency_options = dependency_options + [(f"T{task_id}: {default_name[:20]}...", task_id)]

gantt_task_rows = [HBox([t["name"], t["duration"], t["dependency"]]) for t in task_widgets]

# --- (MODIFICADO) Botones específicos para la pestaña de Arranque ---
gantt_button = widgets.Button(
    description='Generar Gantt (Timeline)',
    button_style='info',
    icon='chart-bar',
    layout=Layout(width='45%', margin='10px 0 0 0')
)
gantt_output = widgets.Output() # Salida para el gráfico Gantt

# --- (NUEVO) Botón y salida para el Diagrama de Red (PERT) ---
gantt_pert_button = widgets.Button(
    description='Generar Red (Dependencias)',
    button_style='warning', # Botón naranja
    icon='project-diagram',
    layout=Layout(width='45%', margin='10px 0 0 0')
)
gantt_pert_output = widgets.Output() # Salida para el gráfico Graphviz

# --- (NUEVO) Agrupar botones de Gantt ---
gantt_buttons_box = HBox([gantt_button, gantt_pert_button], layout=Layout(width='90%', justify_content='space-around'))

# --- Simulación de Progreso ---
progress_sliders = []
progress_bars = []
progress_ui_box = VBox()

# --- (MODIFICADO) Ensamblado de la pestaña F ---
tab_arranque = VBox([
    widgets.HTML("<h3>Definición de la Secuencia de Arranque (Dead Ship)</h3>"),
    gantt_header,
    *gantt_task_rows,
    gantt_buttons_box, # Botones (Gantt y PERT)
    gantt_output,      # Salida Gantt
    gantt_pert_output, # Salida PERT
    widgets.HTML("<h3 style='margin-top: 20px;'>Simulación de Progreso</h3>"),
    progress_ui_box
])

# --- Ensamblado de la Interfaz (UI) ---
tab_ui = widgets.Tab()
tab_ui.children = [accordion_comb, accordion_lub, accordion_agua, accordion_aire, accordion_achique, tab_arranque]
tab_ui.set_title(0, 'Bloque A: Combustible'); tab_ui.set_title(1, 'Bloque B: Lubricación')
tab_ui.set_title(2, 'Bloque C: Agua Dulce'); tab_ui.set_title(3, 'Bloque D: Aire Comprimido')
tab_ui.set_title(4, 'Bloque E: Achique'); tab_ui.set_title(5, 'Bloque F: Arranque')

# --- Botones y Salidas GLOBALES (para Bloques A-E) ---
run_button = widgets.Button(
    description='Ejecutar Análisis (Log Detallado)', button_style='success',
    tooltip='Haga clic para calcular y mostrar el log de texto completo',
    icon='cogs', layout=Layout(width='45%')
)
output_area = widgets.Output()

reporte_md_button = widgets.Button(
    description='Generar Reporte Markdown', button_style='primary',
    tooltip='Haga clic para generar un resumen dinámico en Markdown',
    icon='file-text', layout=Layout(width='45%')
)
reporte_md_output = widgets.HTML(
    value="<p>Presione <b>'Generar Reporte Markdown'</b> para ver el resumen de cumplimiento.</p>",
    layout=Layout(width='90%', border='1px solid #ccc', padding='10px', margin='10px 0 0 0')
)


# -------------------------------------------------------------------
# 4. LÓGICA DE EJECUCIÓN (Callback)
# -------------------------------------------------------------------

def get_config_from_ui() -> BuqueConfig:
    """Función helper para recolectar todos los datos de los widgets A-E."""
    return BuqueConfig(
        mpp = MotorPrincipalConfig(w_mpp_potencia.value, w_mpp_tipo_comb.value, w_mpp_sfoc_hfo.value, w_mpp_sfoc_mdo.value, w_mpp_factor_carga.value, w_mpp_vol_barrido.value),
        mmaa = MotorAuxiliarConfig(w_mmaa_potencia.value, w_mmaa_sfoc.value, w_mmaa_numero.value, w_mmaa_en_servicio.value, w_mmaa_factor_carga.value, w_mmaa_vol_barrido.value),
        caldera = CalderaConfig(w_caldera_consumo.value, w_caldera_tipo_comb.value, w_caldera_sfoc_hfo.value, w_caldera_sfoc_mdo.value, w_caldera_sfoc_mgo.value, w_caldera_en_nav.value),
        tanques_comb = TanquesCombustibleConfig(w_tanque_alm_hfo.value, w_tanque_alm_mdo.value, w_tanque_sed_hfo.value, w_tanque_sed_mdo.value, w_tanque_dia_hfo.value, w_tanque_dia_mdo.value),
        densidad_hfo_t_m3=w_densidad_hfo.value, densidad_mdo_t_m3=w_densidad_mdo.value,
        criterios_lub = CriteriosLubricacionConfig(w_lub_crit_carter_L_kW.value, w_lub_crit_carter_mmaa_m3.value, w_lub_crit_almac_mpp.value, w_lub_crit_almac_mmaa.value, w_lub_crit_sucio_mpp.value, w_lub_crit_sucio_mmaa.value),
        tanques_lub = TanquesLubricacionConfig(w_lub_tanq_circ_mpp.value, w_lub_tanq_almac_mpp.value, w_lub_tanq_almac_mmaa.value, w_lub_tanq_sucio_mpp.value, w_lub_tanq_sucio_mmaa.value),
        criterios_agua = CriteriosAguaDulceConfig(w_agua_crit_tripulacion.value, w_agua_crit_dotacion.value, w_agua_crit_autonomia.value, w_agua_crit_reserva_tec.value),
        sistema_agua = SistemaAguaDulceConfig(w_agua_proy_generador.value, w_agua_proy_tanques.value),
        criterios_aire = CriteriosAireComprimidoConfig(w_aire_crit_mpp_arr.value, w_aire_crit_mmaa_arr.value, w_aire_crit_k_mpp.value, w_aire_crit_k_mmaa.value, w_aire_crit_pmax.value, w_aire_crit_pmin_mpp.value, w_aire_crit_pmin_mmaa.value),
        sistema_aire = SistemaAireComprimidoConfig(w_aire_proy_num.value, w_aire_proy_vol.value),
        criterios_achique = CriteriosAchiqueConfig(w_achique_crit_k1_hfo.value, w_achique_crit_k1_mdo.value, w_achique_crit_dias.value),
        sistema_achique = SistemaAchiqueConfig(w_achique_proy_fangos.value, w_achique_proy_sentina.value)
    )

# --- Callback para el botón de LOG DETALLADO (A-E) ---
def run_analysis_full(b):
    with output_area:
        clear_output(wait=True); print("Iniciando análisis detallado... Por favor, espere.")
        try:
            config = get_config_from_ui()
            print("Iniciando análisis del Módulo A (Combustible)...")
            analisis_comb = AnalisisSistemaCombustible(config); analisis_comb.generar_reporte()
            print("\nIniciando análisis del Módulo B (Lubricación)...")
            analisis_lub = AnalisisSistemaLubricacion(config); analisis_lub.generar_reporte()
            print("\nIniciando análisis del Módulo C (Agua Dulce)...")
            analisis_agua = AnalisisSistemaAguaDulce(config); analisis_agua.generar_reporte()
            print("\nIniciando análisis del Módulo D (Aire Comprimido)...")
            analisis_aire = AnalisisSistemaAireComprimido(config); analisis_aire.generar_reporte()
            print("\nIniciando análisis del Módulo E (Achique/Fangos)...")
            analisis_achique = AnalisisSistemaAchique(config, analisis_comb.consumos_diarios); analisis_achique.generar_reporte()
            print("\n\n=======================================================================\n ANÁLISIS COMPLETO (A, B, C, D, E) FINALIZADO\n=======================================================================")
        except ValueError as ve: print(f"\nERROR DE VALIDACIÓN: {ve}")
        except Exception as e: print(f"\nERROR INESPERADO DURANTE EL ANÁLISIS: {e}")

# --- Callback para el REPORTE MARKDOWN (A-E) ---
def generar_reporte_markdown_callback(b):
    try:
        reporte_md_output.value = "<h4>Generando reporte resumen...</h4>"
        config = get_config_from_ui()
        analisis_comb = AnalisisSistemaCombustible(config); analisis_lub = AnalisisSistemaLubricacion(config)
        analisis_agua = AnalisisSistemaAguaDulce(config); analisis_aire = AnalisisSistemaAireComprimido(config)
        analisis_achique = AnalisisSistemaAchique(config, analisis_comb.consumos_diarios)
        md_string = "# 📋 Resumen de Cumplimiento de Sistemas\n*Este es un resumen dinámico de los principales criterios.*\n\n"
        md_string += analisis_comb.generar_markdown(); md_string += analisis_lub.generar_markdown()
        md_string += analisis_agua.generar_markdown(); md_string += analisis_aire.generar_markdown()
        md_string += analisis_achique.generar_markdown()
        html_output = markdown.markdown(md_string, extensions=['markdown.extensions.tables'])
        reporte_md_output.value = html_output
    except ValueError as ve: reporte_md_output.value = f"<h3 style='color:red;'>ERROR DE VALIDACIÓN:</h3><pre>{ve}</pre>"
    except Exception as e: reporte_md_output.value = f"<h3 style='color:red;'>ERROR INESPERADO:</h3><pre>{e}</pre>"


# --- Callback para el GANTT (Bloque F) ---
def generate_gantt_callback(b):
    global progress_sliders, progress_bars, progress_ui_box
    with gantt_output:
        clear_output(wait=True); print("Calculando secuencia y generando gráfico Gantt...")
        tasks_data = []; finish_times = {0: 0}

        # 1. Calcular tiempos de inicio y fin (Corregido con 'task_row')
        for i, task_row in enumerate(task_widgets):
            task_id = task_row["id"]; task_name = task_row["name"].value
            task_duration = task_row["duration"].value; dependency_id = task_row["dependency"].value
            if not task_name or task_duration <= 0: continue
            start_minute = finish_times.get(dependency_id, 0); finish_minute = start_minute + task_duration
            finish_times[task_id] = finish_minute
            tasks_data.append(dict(Task=task_name, Start=pd.Timestamp('2025-01-01') + pd.to_timedelta(start_minute, unit='m'), Finish=pd.Timestamp('2025-01-01') + pd.to_timedelta(finish_minute, unit='m'), Duration_Min=task_duration))
        if not tasks_data: print("No se definieron tareas válidas."); return
        df = pd.DataFrame(tasks_data)

        # 2. Crear Gráfico Gantt (Plotly)
        fig = go.Figure(go.Bar(
            y=df['Task'], x=df['Duration_Min'], base=df['Start'], orientation='h',
            text=df['Duration_Min'].astype(str) + " min", textposition='inside', insidetextanchor='middle',
            marker=dict(color='rgba(26, 118, 255, 0.6)', line=dict(color='rgba(26, 118, 255, 1.0)', width=1))
        ))
        fig.update_layout(
            title='Diagrama de Gantt: Secuencia de Arranque (Tiempos Requeridos)',
            xaxis_title='Tiempo (Minutos desde Inicio)', yaxis_title='Etapa',
            yaxis_autorange='reversed', height=300 + (len(df) * 35),
            xaxis=dict(tickmode='linear', tick0=0, dtick=5, gridcolor='rgba(0,0,0,0.1)'),
            plot_bgcolor='rgba(0,0,0,0)'
        )
        fig.update_xaxes(tickformat="%M", range=[pd.Timestamp('2025-01-01'), df['Finish'].max() + pd.to_timedelta(2, unit='m')])
        print(f"Secuencia calculada. Tiempo total requerido: {df['Finish'].max().minute} minutos.")
        fig.show()

        # 3. Actualizar las barras de progreso dinámicamente
        progress_sliders.clear(); progress_bars.clear(); new_progress_widgets = []
        for i, row in df.iterrows():
            slider = widgets.FloatSlider(value=0, min=0, max=100, step=1, description=f'{row["Task"]}:', layout=Layout(width='90%'), style={'description_width': '350px'})
            progress_bar = widgets.FloatProgress(value=0, min=0, max=100, bar_style='info', orientation='horizontal', layout=Layout(width='90%'))
            widgets.jslink((slider, 'value'), (progress_bar, 'value'))
            progress_sliders.append(slider); progress_bars.append(progress_bar)
            new_progress_widgets.append(VBox([slider, progress_bar]))
        progress_ui_box.children = tuple(new_progress_widgets)


# -------------------------------------------------------------------
# 5. MOSTRAR LA APLICACIÓN
# -------------------------------------------------------------------

# Conectar callbacks a botones
run_button.on_click(run_analysis_full)
reporte_md_button.on_click(generar_reporte_markdown_callback)
gantt_button.on_click(generate_gantt_callback)
gantt_pert_button.on_click(generate_pert_callback) # <-- NUEVO

# Agrupar botones GLOBALES (A-E)
controles_botones = HBox([run_button, reporte_md_button], layout=Layout(width='90%', justify_content='space-around'))

# Mostrar la UI completa
display(tab_ui, controles_botones, reporte_md_output, output_area)

HTML(value="<p>Presione <b>'Generar Reporte Markdown'</b> para ver el resumen de cumplimiento.</p>", layout=La…

Output()